### MST AIDS 2023-2024 (Département Génie Informatique)
### Subject : The main purpose behind this lab is to get familiar with Scraping and NLP Pipeline.
### Realize by : Chibani Fahd
### web sourcest : Aljarida24r
### Modèl : NLP

#### PART 1 : Scraping and storing data 

#### I Used Beautiful Soup to scrap multiple pages from Aljarida24  web site and pymongo to connete by Mongodb and store data..

In [ ]:
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['aljarida24']
collection = db['articles']

# URL of the main page
main_url = 'https://aljarida24.ma/'

# Request the main page
main_page = requests.get(main_url)

# Parse the main page
main_soup = BeautifulSoup(main_page.text, 'html.parser')

# Find all the links on the main page that contain "aljarida24.ma/p"
all_links = main_soup.find_all('a', href=True)

# Initialize a set to store unique URLs
unique_urls = set()

# Extract and filter unique URLs from the links
for link in all_links:
    url = link.get('href')
    if url and 'aljarida24.ma/p' in url:
        unique_urls.add(url)
        if len(unique_urls) >= 100:
            break  # Exit loop if we have collected 100 unique URLs

# Loop through unique URLs
for url in unique_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    para_div = soup.find('div', {'class': 'content-text'})
    if para_div:
        # Extract header if available
        header = soup.find('h1')
        if header:
            print("URL:", url)
            print("Header:", header.text.strip())
            header_text = header.text.strip()
        
        # Initialize a variable to store all paragraphs
        all_paragraphs = ""

        # Extract paragraphs
        for item in para_div.contents:
            if item.name == 'p':
                all_paragraphs += item.text.strip() + "  "

        print("Paragraphs:", all_paragraphs)
        print("\n")  # Add a new line for separation between articles
        # Insert data into MongoDB
        article_data = {
            "url": url,
            "header": header_text,
            "paragraphs": all_paragraphs
        }
        
        collection.insert_one(article_data)
